In [1]:
!pip install whisper-openai transformers datasets librosa torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
import os
import whisper

drive.mount('/content/drive')

dataset_path = "/content/drive/MyDrive/task_1"
model = whisper.load_model("tiny")

Mounted at /content/drive


100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 118MiB/s]


In [4]:
import librosa
import numpy as np

def preprocess_audio(audio_path):
    audio, sr = librosa.load(audio_path, sr=16000, mono=True)
    return audio

In [9]:
# Explore the complete directory structure
import os

def explore_directory(path, level=0):
    indent = "  " * level
    for item in os.listdir(path):
        item_path = os.path.join(path, item)
        if os.path.isdir(item_path):
            print(f"{indent}📁 {item}/")
            explore_directory(item_path, level + 1)
        else:
            print(f"{indent}📄 {item}")

explore_directory("/content/drive/MyDrive/task_1")

📄 sand_task_1.xlsx
📄 .DS_Store
📁 training/
  📁 phonationA/
    📄 ID051_phonationA.wav
    📄 ID023_phonationA.wav
    📄 ID169_phonationA.wav
    📄 ID239_phonationA.wav
    📄 ID098_phonationA.wav
    📄 ID192_phonationA.wav
    📄 ID155_phonationA.wav
    📄 ID141_phonationA.wav
    📄 ID093_phonationA.wav
    📄 ID296_phonationA.wav
    📄 ID181_phonationA.wav
    📄 ID218_phonationA.wav
    📄 ID172_phonationA.wav
    📄 ID067_phonationA.wav
    📄 ID116_phonationA.wav
    📄 ID319_phonationA.wav
    📄 ID186_phonationA.wav
    📄 ID174_phonationA.wav
    📄 ID109_phonationA.wav
    📄 ID326_phonationA.wav
    📄 ID262_phonationA.wav
    📄 ID287_phonationA.wav
    📄 ID233_phonationA.wav
    📄 ID096_phonationA.wav
    📄 ID072_phonationA.wav
    📄 ID204_phonationA.wav
    📄 ID047_phonationA.wav
    📄 ID195_phonationA.wav
    📄 ID059_phonationA.wav
    📄 ID126_phonationA.wav
    📄 ID222_phonationA.wav
    📄 ID212_phonationA.wav
    📄 ID275_phonationA.wav
    📄 ID201_phonationA.wav
    📄 ID207_phonationA.

In [11]:
from tqdm.auto import tqdm
import json
import os

dataset_path = "/content/drive/MyDrive/task_1/training"

# Get all subfolders
subfolders = ['phonationA', 'phonationE', 'phonationI', 'phonationU', 'phonationO', 'rhythmKA', 'rhythmTA', 'rhythmPA']

transcriptions = []
wav_files = []

# Collect all WAV files from all subfolders
for folder in subfolders:
    folder_path = os.path.join(dataset_path, folder)
    if os.path.exists(folder_path):
        folder_wavs = [os.path.join(folder, f) for f in os.listdir(folder_path) if f.lower().endswith('.wav')]
        wav_files.extend(folder_wavs)
        print(f"Found {len(folder_wavs)} WAV files in {folder}")

print(f"Total WAV files found: {len(wav_files)}")

if len(wav_files) == 0:
    print("No WAV files found in any subfolder!")
else:
    print("Transcribing audio files...")
    for file_path in tqdm(wav_files, desc="Processing"):
        try:
            full_audio_path = os.path.join(dataset_path, file_path)
            result = model.transcribe(full_audio_path)
            transcriptions.append({
                "file": file_path,
                "text": result["text"].strip()
            })
        except Exception as e:
            print(f"Error with {file_path}: {str(e)}")

print(f"Successfully transcribed {len(transcriptions)} files")

Found 272 WAV files in phonationA
Found 272 WAV files in phonationE
Found 272 WAV files in phonationI
Found 272 WAV files in phonationU
Found 272 WAV files in phonationO
Found 272 WAV files in rhythmKA
Found 272 WAV files in rhythmTA
Found 272 WAV files in rhythmPA
Total WAV files found: 2176
Transcribing audio files...


Processing:   0%|          | 0/2176 [00:00<?, ?it/s]

Successfully transcribed 2176 files


In [12]:
output_file = "/content/drive/MyDrive/task_1_transcriptions.json"
with open(output_file, 'w') as f:
    json.dump(transcriptions, f, indent=2)

print(f"Transcription complete. Saved {len(transcriptions)} files to {output_file}")

Transcription complete. Saved 2176 files to /content/drive/MyDrive/task_1_transcriptions.json


In [4]:
# Define dataset_path again (it might be lost from previous execution)
dataset_path = "/content/drive/MyDrive/task_1/training"

# Load your existing transcriptions (skip re-transcription)
transcriptions_file = "/content/drive/MyDrive/task_1_transcriptions.json"
with open(transcriptions_file, 'r') as f:
    transcriptions = json.load(f)

print(f"Loaded {len(transcriptions)} existing transcriptions")

# Create training dataset
from datasets import Dataset

def create_training_data(transcriptions):
    data = []
    for item in transcriptions:
        audio_path = os.path.join(dataset_path, item["file"])
        if os.path.exists(audio_path):
            data.append({
                "audio": audio_path,
                "transcription": item["text"]
            })
    return Dataset.from_list(data)

train_dataset = create_training_data(transcriptions)
print(f"Training dataset created with {len(train_dataset)} samples")

Loaded 2176 existing transcriptions
Training dataset created with 2176 samples


In [6]:
# Load model and processor for fine-tuning
from transformers import WhisperForConditionalGeneration, WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

In [2]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

print(" Loading processor and model...")
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
print(" Processor and model loaded!")

 Loading processor and model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


 Processor and model loaded!


In [ ]:
# COMPLETE TRAINING CODE
import os
import json
from datasets import Dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from tqdm.auto import tqdm
import librosa
import numpy as np
import torch

print(" STARTING COMPLETE TRAINING PROCESS...")

# 1. Load transcriptions
print(" Loading transcriptions...")
transcriptions_file = "/content/drive/MyDrive/task_1_transcriptions.json"
with open(transcriptions_file, 'r') as f:
    transcriptions = json.load(f)
print(f" Loaded {len(transcriptions)} transcriptions")

# 2. Load processor and model
print(" Loading model and processor...")
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

# 3. Create dataset with ALL files
print(" Creating training dataset...")
dataset_path = "/content/drive/MyDrive/task_1/training"

processed_data = []
for i in tqdm(range(len(transcriptions)), desc=" Processing files"):
    try:
        item = transcriptions[i]
        audio_path = os.path.join(dataset_path, item["file"])

        # Load audio
        audio_array, sampling_rate = librosa.load(audio_path, sr=16000, mono=True)

        # Process audio features
        input_features = processor.feature_extractor(
            audio_array,
            sampling_rate=16000,
            return_tensors="pt"
        ).input_features[0]

        # Process text labels
        labels = processor.tokenizer(
            item["text"],
            padding=False,
            truncation=True,
            max_length=448
        ).input_ids

        processed_data.append({
            "input_features": input_features,
            "labels": labels
        })

    except Exception as e:
        continue

final_dataset = Dataset.from_list(processed_data)
print(f" Dataset created: {len(final_dataset)} samples")

# 4. Setup and start training
print(" Setting up training...")
data_collator = DataCollatorForSeq2Seq(
    tokenizer=processor.tokenizer,
    model=model,
    padding=True
)

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-tiny-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=500,
    fp16=True,
    save_strategy="steps",
    save_steps=100,
    logging_steps=25,
    report_to="none",
    push_to_hub=False,
    remove_unused_columns=False,
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=final_dataset,
    data_collator=data_collator,
    processing_class=processor.feature_extractor,
)

print(" Starting training...")
trainer.train()

# 5. Save model
print(" Saving model...")
trainer.save_model("/content/drive/MyDrive/whisper-tiny-finetuned-task1")
processor.save_pretrained("/content/drive/MyDrive/whisper-tiny-finetuned-task1")
print(" TRAINING COMPLETED SUCCESSFULLY!")

 STARTING COMPLETE TRAINING PROCESS...
 Loading transcriptions...
 Loaded 2176 transcriptions
 Loading model and processor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


 Creating training dataset...


 Processing files:   0%|          | 0/2176 [00:00<?, ?it/s]

In [18]:
# CREATE GROUND TRUTH AND CALCULATE REAL F1 SCORES
import json
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

# Load your predictions
with open("/content/drive/MyDrive/task_1_transcriptions.json", 'r') as f:
    predictions_data = json.load(f)

print(f"📊 Total files: {len(predictions_data)}")

# Create sample ground truth based on common phonation patterns
GROUND_TRUTH = {
    # Phonation files - usually single vowels or simple sounds
    "phonationA/": "aaa",
    "phonationE/": "eee",
    "phonationI/": "iii",
    "phonationO/": "ooo",
    "phonationU/": "uuu",

    # Rhythm files - usually consonant-vowel patterns
    "rhythmKA/": "kaka",
    "rhythmTA/": "tata",
    "rhythmPA/": "papa"
}

# Calculate real metrics
predictions = []
references = []

for item in predictions_data:
    pred_text = item["text"].strip().lower()
    predictions.append(pred_text)

    # Determine ground truth based on folder
    file_path = item["file"]
    ground_truth_text = "unknown"

    for folder, truth in GROUND_TRUTH.items():
        if folder in file_path:
            ground_truth_text = truth
            break

    references.append(ground_truth_text)

# Calculate metrics
exact_matches = [1 if pred == ref else 0 for pred, ref in zip(predictions, references)]
accuracy = np.mean(exact_matches)

y_true = [1] * len(predictions)
y_pred = [1 if pred == ref else 0 for pred, ref in zip(predictions, references)]
f1 = f1_score(y_true, y_pred)

print(f"🎯 REAL METRICS WITH GROUND TRUTH:")
print(f"✅ ACCURACY: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"✅ F1 SCORE: {f1:.4f}")
print(f"📊 FILES EVALUATED: {len(predictions)}")
print(f"✅ CORRECT: {sum(exact_matches)}")
print(f"❌ INCORRECT: {len(predictions) - sum(exact_matches)}")

# Show examples with comparisons
print(f"\n🔍 SAMPLE COMPARISONS:")
for i in range(10):
    status = "✅" if predictions[i] == references[i] else "❌"
    print(f"File: {predictions_data[i]['file']}")
    print(f"  Prediction: '{predictions[i]}'")
    print(f"  Ground Truth: '{references[i]}'")
    print(f"  Status: {status}\n")

📊 Total files: 2176
🎯 REAL METRICS WITH GROUND TRUTH:
✅ ACCURACY: 0.0005 (0.05%)
✅ F1 SCORE: 0.0009
📊 FILES EVALUATED: 2176
✅ CORRECT: 1
❌ INCORRECT: 2175

🔍 SAMPLE COMPARISONS:
File: phonationA/ID051_phonationA.wav
  Prediction: 's��'
  Ground Truth: 'aaa'
  Status: ❌

File: phonationA/ID023_phonationA.wav
  Prediction: 'i'
  Ground Truth: 'aaa'
  Status: ❌

File: phonationA/ID169_phonationA.wav
  Prediction: 'ave.'
  Ground Truth: 'aaa'
  Status: ❌

File: phonationA/ID239_phonationA.wav
  Prediction: ''
  Ground Truth: 'aaa'
  Status: ❌

File: phonationA/ID098_phonationA.wav
  Prediction: 'karnan- после'
  Ground Truth: 'aaa'
  Status: ❌

File: phonationA/ID192_phonationA.wav
  Prediction: 'sides'
  Ground Truth: 'aaa'
  Status: ❌

File: phonationA/ID155_phonationA.wav
  Prediction: 'yeah ah'
  Ground Truth: 'aaa'
  Status: ❌

File: phonationA/ID141_phonationA.wav
  Prediction: 'oh'
  Ground Truth: 'aaa'
  Status: ❌

File: phonationA/ID093_phonationA.wav
  Prediction: ''
  Ground Tru